# Log Analysis with DuckDB
This notebook demonstrates how to use DuckDB to analyze audit and signin logs stored in Parquet files. We will:
1. Load the Parquet files into DuckDB tables.
2. Inspect the schema and contents of these tables.
3. Merge these tables to create an `events` table.
4. Create a `sessions` table by aggregating the `events` table.
5. Showcase the results.

In [1]:
# Import necessary libraries
import duckdb
import os
import pandas as pd

## Step 1: Load Parquet Files into DuckDB Tables

In [2]:
# Directory where Parquet files are stored
auditlogs_dir = "./parquet_data/insights-logs-auditlogs"
signinlogs_dir = "./parquet_data/insights-logs-signinlogs"

# Initialize DuckDB connection
con = duckdb.connect("azure_logs.db")

### Load data from Parquet files

In [3]:
auditlogs_combined_file = os.path.join(auditlogs_dir, "combined_table.parquet")
signinlogs_combined_file = os.path.join(signinlogs_dir, "combined_table.parquet")

# Drop existing tables if they exist
con.execute("DROP TABLE IF EXISTS combined_table_auditlogs")
con.execute("DROP TABLE IF EXISTS combined_table_signinlogs")
con.execute("DROP TABLE IF EXISTS events")
con.execute("DROP TABLE IF EXISTS sessions")

# Create tables from Parquet files
con.execute(f"CREATE TABLE combined_table_auditlogs AS SELECT * FROM read_parquet('{auditlogs_combined_file}')")
con.execute(f"CREATE TABLE combined_table_signinlogs AS SELECT * FROM read_parquet('{signinlogs_combined_file}')")

## Step 2: Inspect the Tables
Let's inspect the schema and first few rows of each table.

In [4]:
# Describe the schema of the audit logs table
auditlogs_schema_df = con.execute("DESCRIBE SELECT * FROM combined_table_auditlogs").df()
auditlogs_schema_df

,column_name,column_type,null,key,default,extra
0,time,VARCHAR,YES,None,None,None
1,resourceId,VARCHAR,YES,None,None,None
2,operationName,VARCHAR,YES,None,None,None
3,operationVersion,VARCHAR,YES,None,None,None
4,category,VARCHAR,YES,None,None,None
5,tenantId,VARCHAR,YES,None,None,None
6,resultSignature,VARCHAR,YES,None,None,None
7,durationMs,BIGINT,YES,None,None,None
8,correlationId,VARCHAR,YES,None,None,None
9,identity,VARCHAR,YES,None,None,None


In [5]:
# Display the first 10 rows of the audit logs table
auditlogs_sample_df = con.execute("SELECT * FROM combined_table_auditlogs LIMIT 10").df()
auditlogs_sample_df

,time,resourceId,operationName,operationVersion,category,tenantId,resultSignature,durationMs,correlationId,identity,...,properties.targetResources.groupType,properties.additionalDetails.key,properties.additionalDetails.value,properties.additionalDetails,properties.initiatedBy.user.roles,properties.targetResources.administrativeUnits,properties.targetResources.modifiedProperties.displayName,properties.targetResources.modifiedProperties.newValue,properties.targetResources.modifiedProperties.oldValue,properties.targetResources.modifiedProperties
0,2024-07-04T01:10:45.6603764Z,/tenants/7c86bd1f-6161-48e1-ae9c-9cd406835b6f/...,Add service principal,1.0,AuditLogs,7c86bd1f-6161-48e1-ae9c-9cd406835b6f,None,0,89dd2ef6-6eff-4d1c-8b68-8d1966618761,Microsoft Azure AD Internal - Jit Provisioning,...,None,AppId,95de633a-083e-42f5-b444-a4295d8e9314,NaN,NaN,NaN,AccountEnabled,[true],[],NaN
1,2024-07-04T01:10:45.6603764Z,/tenants/7c86bd1f-6161-48e1-ae9c-9cd406835b6f/...,Add service principal,1.0,AuditLogs,7c86bd1f-6161-48e1-ae9c-9cd406835b6f,None,0,89dd2ef6-6eff-4d1c-8b68-8d1966618761,Microsoft Azure AD Internal - Jit Provisioning,...,None,AppId,95de633a-083e-42f5-b444-a4295d8e9314,NaN,NaN,NaN,AppAddress,"[{""AddressType"":0,""Address"":""https://whiteboar...",[],NaN
2,2024-07-04T01:10:45.6603764Z,/tenants/7c86bd1f-6161-48e1-ae9c-9cd406835b6f/...,Add service principal,1.0,AuditLogs,7c86bd1f-6161-48e1-ae9c-9cd406835b6f,None,0,89dd2ef6-6eff-4d1c-8b68-8d1966618761,Microsoft Azure AD Internal - Jit Provisioning,...,None,AppId,95de633a-083e-42f5-b444-a4295d8e9314,NaN,NaN,NaN,AppPrincipalId,"[""95de633a-083e-42f5-b444-a4295d8e9314""]",[],NaN
3,2024-07-04T01:10:45.6603764Z,/tenants/7c86bd1f-6161-48e1-ae9c-9cd406835b6f/...,Add service principal,1.0,AuditLogs,7c86bd1f-6161-48e1-ae9c-9cd406835b6f,None,0,89dd2ef6-6eff-4d1c-8b68-8d1966618761,Microsoft Azure AD Internal - Jit Provisioning,...,None,AppId,95de633a-083e-42f5-b444-a4295d8e9314,NaN,NaN,NaN,DisplayName,"[""Microsoft Whiteboard Services""]",[],NaN
4,2024-07-04T01:10:45.6603764Z,/tenants/7c86bd1f-6161-48e1-ae9c-9cd406835b6f/...,Add service principal,1.0,AuditLogs,7c86bd1f-6161-48e1-ae9c-9cd406835b6f,None,0,89dd2ef6-6eff-4d1c-8b68-8d1966618761,Microsoft Azure AD Internal - Jit Provisioning,...,None,AppId,95de633a-083e-42f5-b444-a4295d8e9314,NaN,NaN,NaN,ServicePrincipalName,"[""https://whiteboard.microsoft.com"",""95de633a-...",[],NaN
5,2024-07-04T01:10:45.6603764Z,/tenants/7c86bd1f-6161-48e1-ae9c-9cd406835b6f/...,Add service principal,1.0,AuditLogs,7c86bd1f-6161-48e1-ae9c-9cd406835b6f,None,0,89dd2ef6-6eff-4d1c-8b68-8d1966618761,Microsoft Azure AD Internal - Jit Provisioning,...,None,AppId,95de633a-083e-42f5-b444-a4295d8e9314,NaN,NaN,NaN,Credential,"[{""CredentialType"":2,""KeyStoreId"":""291154f0-a9...",[],NaN
6,2024-07-04T01:10:45.6603764Z,/tenants/7c86bd1f-6161-48e1-ae9c-9cd406835b6f/...,Add service principal,1.0,AuditLogs,7c86bd1f-6161-48e1-ae9c-9cd406835b6f,None,0,89dd2ef6-6eff-4d1c-8b68-8d1966618761,Microsoft Azure AD Internal - Jit Provisioning,...,None,AppId,95de633a-083e-42f5-b444-a4295d8e9314,NaN,NaN,NaN,Included Updated Properties,"""AccountEnabled, AppAddress, AppPrincipalId, D...",None,NaN
7,2024-07-04T01:10:45.6603764Z,/tenants/7c86bd1f-6161-48e1-ae9c-9cd406835b6f/...,Add service principal,1.0,AuditLogs,7c86bd1f-6161-48e1-ae9c-9cd406835b6f,None,0,89dd2ef6-6eff-4d1c-8b68-8d1966618761,Microsoft Azure AD Internal - Jit Provisioning,...,None,AppId,95de633a-083e-42f5-b444-a4295d8e9314,NaN,NaN,NaN,TargetId.ServicePrincipalNames,"""https://whiteboard.microsoft.com;95de633a-083...",None,NaN
8,2024-07-04T01:29:39.0447402Z,/tenants/7c86bd1f-6161-48e1-ae9c-9cd406835b6f/...,Update user,1.0,AuditLogs,7c86bd1f-6161-48e1-ae9c-9cd406835b6f,None,0,510aa855-0fe7-4620-8c4d-72b112626c28,Azure MFA StrongAuthenticationService,...,None,UserType,Member,NaN,NaN,NaN,StrongAuthenticationPhoneAppDetail,"[{""DeviceName"":""iPhone"",""DeviceToken"":""apns2-0...","[{""DeviceName"":""iPhone"",""DeviceToken"":""apns2-

In [6]:
# Describe the schema of the signin logs table
signinlogs_schema_df = con.execute("DESCRIBE SELECT * FROM combined_table_signinlogs").df()
signinlogs_schema_df

,column_name,column_type,null,key,default,extra
0,time,VARCHAR,YES,None,None,None
1,resourceId,VARCHAR,YES,None,None,None
2,operationName,VARCHAR,YES,None,None,None
3,operationVersion,VARCHAR,YES,None,None,None
4,category,VARCHAR,YES,None,None,None
...,...,...,...,...,...,...
106,properties.conditionalAccessAudiences.applicat...,VARCHAR,YES,None,None,None
107,properties.conditionalAccessAudiences.conditio...,VARCHAR,YES,None,None,None
108,properties.conditionalAccessAudiences,INTEGER,YES,None,None,None
109,properties.appliedConditionalAccessPolicies.en...,VARCHAR,YES,None,None,None


In [7]:
# Display the first 10 rows of the signin logs table
signinlogs_sample_df = con.execute("SELECT * FROM combined_table_signinlogs LIMIT 10").df()
signinlogs_sample_df

,time,resourceId,operationName,operationVersion,category,tenantId,resultType,resultSignature,durationMs,callerIpAddress,...,properties.authenticationRequirementPolicies,properties.sessionLifetimePolicies,properties.sessionLifetimePolicies.detail,properties.sessionLifetimePolicies.expirationRequirement,properties.authenticationStrengths,properties.conditionalAccessAudiences.applicationId,properties.conditionalAccessAudiences.conditionalAccessAudienceReason,properties.conditionalAccessAudiences,properties.appliedConditionalAccessPolicies.enforcedGrantControls,properties.appliedConditionalAccessPolicies.enforcedSessionControls
0,2024-07-27T00:43:55.1493800Z,/tenants/7c86bd1f-6161-48e1-ae9c-9cd406835b6f/...,Sign-in activity,1.0,SignInLogs,7c86bd1f-6161-48e1-ae9c-9cd406835b6f,0,None,0,98.151.140.122,...,NaN,NaN,None,None,NaN,03456469-42aa-43cb-915d-a070c9300e4d,confidentialClientIdToken,NaN,Mfa,NaN
1,2024-07-27T00:43:55.1493800Z,/tenants/7c86bd1f-6161-48e1-ae9c-9cd406835b6f/...,Sign-in activity,1.0,SignInLogs,7c86bd1f-6161-48e1-ae9c-9cd406835b6f,0,None,0,98.151.140.122,...,NaN,NaN,None,None,NaN,03456469-42aa-43cb-915d-a070c9300e4d,confidentialClientIdToken,NaN,Mfa,NaN
2,2024-07-27T00:43:55.1493800Z,/tenants/7c86bd1f-6161-48e1-ae9c-9cd406835b6f/...,Sign-in activity,1.0,SignInLogs,7c86bd1f-6161-48e1-ae9c-9cd406835b6f,0,None,0,98.151.140.122,...,NaN,NaN,None,None,NaN,03456469-42aa-43cb-915d-a070c9300e4d,confidentialClientIdToken,NaN,Mfa,NaN
3,2024-07-27T00:43:55.1493800Z,/tenants/7c86bd1f-6161-48e1-ae9c-9cd406835b6f/...,Sign-in activity,1.0,SignInLogs,7c86bd1f-6161-48e1-ae9c-9cd406835b6f,0,None,0,98.151.140.122,...,NaN,NaN,None,None,NaN,03456469-42aa-43cb-915d-a070c9300e4d,confidentialClientIdToken,NaN,Mfa,NaN
4,2024-07-27T02:58:09.6616781Z,/tenants/7c86bd1f-6161-48e1-ae9c-9cd406835b6f/...,Sign-in activity,1.0,SignInLogs,7c86bd1f-6161-48e1-ae9c-9cd406835b6f,0,None,0,2601:647:6881:93d0:526:24ce:ca74:bb15,...,NaN,NaN,None,None,NaN,03456469-42aa-43cb-915d-a070c9300e4d,confidentialClientIdToken,NaN,None,NaN
5,2024-07-27T02:58:09.6616781Z,/tenants/7c86bd1f-6161-48e1-ae9c-9cd406835b6f/...,Sign-in activity,1.0,SignInLogs,7c86bd1f-6161-48e1-ae9c-9cd406835b6f,0,None,0,2601:647:6881:93d0:526:24ce:ca74:bb15,...,NaN,NaN,None,None,NaN,03456469-42aa-43cb-915d-a070c9300e4d,confidentialClientIdToken,NaN,None,NaN
6,2024-07-27T03:39:44.3982838Z,/tenants/7c86bd1f-6161-48e1-ae9c-9cd406835b6f/...,Sign-in activity,1.0,SignInLogs,7c86bd1f-6161-48e1-ae9c-9cd406835b6f,0,None,0,2601:547:600:9490:bd39:ca39:5dbd:3168,...,NaN,NaN,None,None,NaN,797f4846-ba00-4fd7-ba43-dac1f8f63013,unknownFutureValue,NaN,Mfa,NaN
7,2024-07-27T03:39:44.3982838Z,/tenants/7c86bd1f-6161-48e1-ae9c-9cd406835b6f/...,Sign-in activity,1.0,SignInLogs,7c86bd1f-6161-48e1-ae9c-9cd406835b6f,0,None,0,2601:547:600:9490:bd39:ca39:5dbd:3168,...,NaN,NaN,None,None,NaN,797f4846-ba00-4fd7-ba43-dac1f8f63013,unknownFutureValue,NaN,Mfa,NaN
8,2024-07-27T03:39:44.3982838Z,/tenants/7c86bd1f-6161-48e1-ae9c-9cd406835b6f/...,Sign-in activity,1.0,SignInLogs,7c86bd1f-6161-48e1-ae9c-9cd406835b6f,0,None,0,2601:547:600:9490:bd39:ca39:5dbd:3168,...,NaN,NaN,None,None,NaN,797f4846-ba00-4fd7-ba43-dac1f8f63013,unknownFutureValue,NaN,Mfa,NaN
9,2024-07-27T03:39:44.3982838Z,/tenants/7c86bd1f-6161-48e1-ae9c-9cd406835b6f/...,Sign-in activity,1.0,SignInLogs,7c86bd1f-6161-48e1-ae9c-9cd406835b6f,0,None,0,2601:547:600:9490:bd39:ca39:5dbd:3168,...,NaN,NaN,None,None,NaN,797f4846-ba00-4fd7-ba43-dac1f8f63013,unknownFutureValue,NaN,Mfa,NaN


## Step 3: Merge Tables to Create `events` Table
In this step, we will merge the `combined_table_auditlogs` and `combined_table_signinlogs` tables into a single `events` table. We will rename `correlationId` to `exfSessionId` and select the relevant columns from both tables. We will use the `UNION ALL` operator to combine rows from both `combined_table_auditlogs` and `combined_table_signinlogs` into the `events` table. The `DISTINCT` keyword ensures that duplicate rows are removed.

In [8]:
# Drop existing events table if it exists
con.execute("DROP TABLE IF EXISTS events")

# Query to merge the tables
query = '''
CREATE TABLE events AS
SELECT DISTINCT
    "time", 
    "operationName", 
    "category",  
    "correlationId" AS "exfSessionId", 
    "callerIpAddress" AS ipAddress, 
    "properties.initiatedBy.user.id" AS "userId", 
    "properties.initiatedBy.user.userPrincipalName" AS "userPrincipalName", 
    "properties.targetResources.displayName" AS "appDisplayName"
FROM combined_table_auditlogs
UNION ALL
SELECT 
    "time", 
    "operationName", 
    "category", 
    "correlationId" AS "exfSessionId", 
    "callerIpAddress" AS ipAddress, 
    "properties.userId" AS "userId", 
    "properties.userPrincipalName" AS "userPrincipalName", 
    "properties.appDisplayName" AS "appDisplayName"
FROM combined_table_signinlogs;
'''

# Execute the query to create the events table
con.execute(query)

# Verify the results
events_df = con.execute("SELECT * FROM events").df()

# Save to CSV
events_df.to_csv("events.csv", index=False)

events_df

,time,operationName,category,exfSessionId,ipAddress,userId,userPrincipalName,appDisplayName
0,2024-07-04T01:29:39.0447402Z,Update user,AuditLogs,510aa855-0fe7-4620-8c4d-72b112626c28,13.91.147.72,None,None,None
1,2024-07-04T07:22:35.4470100Z,Update application,AuditLogs,2f1871ef-5b8f-4344-9f94-f9540143f9bc,None,a41d7eeb-8e40-4878-990f-98342fdd5abb,lukasz.drozdz@exaforce.com,demo-dev
2,2024-07-17T00:50:59.9363246Z,Update StsRefreshTokenValidFrom Timestamp,AuditLogs,f7d994ab-b15e-4478-b667-578a636cf5c3,None,deab9db2-36f5-45b7-b6e1-74e5e8ba9594,magrawal@exaforce.com,None
3,2024-07-17T00:52:41.9992601Z,Update user,AuditLogs,742d1d66-7963-4924-837d-f9d96a20c1fa,None,deab9db2-36f5-45b7-b6e1-74e5e8ba9594,magrawal@exaforce.com,None
4,2024-07-17T13:20:40.3251811Z,Update user,AuditLogs,1da4bb52-426f-4f72-8cad-05755d22b1c5,None,1258c479-653f-4efc-b074-0615d08a06ef,wbeasley@exaforce.com,None
...,...,...,...,...,...,...,...,...
9142,2024-07-22T21:22:09.4497281Z,Sign-in activity,SignInLogs,2ede8048-4f8e-4f44-866a-2da23b644f4c,2601:547:600:9490:9ccf:268e:8cf3:7c5,c02c5f2b-0197-4b7a-b7e8-6cd0a5e379e8,jpavlik@exaforce.com,demo-dev
9143,2024-07-22T21:34:54.4026949Z,Sign-in activity,SignInLogs,0a16c64b-d273-41e9-9d7b-c9ecec32b47c,50.217.167.134,f39316cd-b04d-448b-8bbb-dbfaf87f3ab1,ahuq@exaforce.com,demo-dev
9144,2024-07-22T21:34:54.4026949Z,Sign-in activity,SignInLogs,0a16c64b-d273-41e9-9d7b-c9ecec32b47c,50.217.167.134,f39316cd-b04d-448b-8bbb-dbfaf87f3ab1,ahuq@exaforce.com,demo-dev
9145,2024-07-22T21:41:56.9341458Z,Sign-in activity,SignInLogs,e657ba2b-a8b6-43d9-8c54-60b02fc594ec,50.217.167.134,9c31208d-9e3e-45f9-a28a-db5baff8c103,kavita@exaforce.com,demo-dev


## Step 4: Create `sessions` Table by Aggregating the `events` Table
In this step, we will create a `sessions` table by aggregating the `events` table based on `exfSessionId`. For each session `exfSessionId`, we will calculate the earliest (`session_start`) and latest (`session_end`) timestamps. We will also determine the most common `userId` and `userPrincipalName` for each session using subqueries with the `GROUP BY` and `ORDER BY` clauses.

In [9]:
# Drop existing sessions table if it exists
con.execute("DROP TABLE IF EXISTS sessions")

# Query to create the sessions table
query3 = '''
CREATE TABLE sessions AS
SELECT
    exfSessionId,
    MIN(CAST("time" AS TIMESTAMP)) AS session_start,
    MAX(CAST("time" AS TIMESTAMP)) AS session_end,
    -- Get the most common values for userId and userPrincipalName
    (SELECT userId FROM events e2 WHERE e1.exfSessionId = e2.exfSessionId GROUP BY userId ORDER BY COUNT(*) DESC LIMIT 1) AS userId,
    (SELECT userPrincipalName FROM events e2 WHERE e1.exfSessionId = e2.exfSessionId GROUP BY userPrincipalName ORDER BY COUNT(*) DESC LIMIT 1) AS userPrincipalName
FROM
    events e1
GROUP BY
    exfSessionId;
'''

con.execute(query3)

# Verify the results
sessions_df = con.execute("SELECT * FROM sessions").df()
# Save to CSV
sessions_df.to_csv("sessions.csv", index=False)
sessions_df

,exfSessionId,session_start,session_end,userId,userPrincipalName
0,f7d994ab-b15e-4478-b667-578a636cf5c3,2024-07-17 00:50:59.935324,2024-07-17 00:50:59.936324,deab9db2-36f5-45b7-b6e1-74e5e8ba9594,magrawal@exaforce.com
1,742d1d66-7963-4924-837d-f9d96a20c1fa,2024-07-17 00:52:41.999260,2024-07-17 00:52:41.999260,deab9db2-36f5-45b7-b6e1-74e5e8ba9594,magrawal@exaforce.com
2,2e5236cd-3113-4346-a629-75899721f3c2,2024-07-18 13:28:39.149599,2024-07-18 13:28:39.231610,f03c445a-4270-4fc6-b9a2-a647b428cb5b,krzysztof.kwapisiewicz@exaforce.com
3,f635bc81-f30c-4413-b862-fa58ad1a3428,2024-07-18 17:42:51.786596,2024-07-18 17:42:51.786596,10e44428-4f3e-4029-911f-e004d2655744,smoy@exaforce.com
4,8cd9e7f0-21b5-4eb5-b132-fb3598333f47,2024-07-30 13:49:43.193218,2024-07-30 13:49:43.193218,cafee0c9-f794-4771-98eb-951abed86970,marek@exaforce.com
...,...,...,...,...,...
3122,adaa03a1-ca56-459b-893c-2e0251a00fdf,2024-07-22 09:40:22.260203,2024-07-22 09:40:22.260203,63dbe567-e92b-4b2a-802c-f561189cd9db,manjunath@exaforce.com
3123,42c86d62-d43d-4472-81da-eb6575dcc24b,2024-07-22 12:28:43.017475,2024-07-22 12:28:43.017475,00c4f81c-6d37-4aa1-abb4-1f18fca4f17f,peter@exaforce.com
3124,991efb7b-3ca9-429a-ac41-c262342d62c8,2024-07-22 14:41:41.371158,2024-07-22 14:41:41.371158,30fa9a67-f74b-4c65-b9a5-c37cee8b6d58,sundar@exaforce.com
3125,e7bda1ef-827d-415a-a1d9-f288dd885ce2,2024-07-22 14:43:58.535114,2024-07-22 14:43:58.535114,d9c3fd33-1c62-471f-938a-9826985f42cf,dresl@exaforce.com


## Step 5: Verify the Created Tables
Let's verify the contents of the `events` and `sessions` tables by checking their schemas and a few sample rows.

In [10]:
# Verify the schema of the events table
events_schema_df = con.execute("DESCRIBE SELECT * FROM events").df()
events_schema_df

,column_name,column_type,null,key,default,extra
0,time,VARCHAR,YES,None,None,None
1,operationName,VARCHAR,YES,None,None,None
2,category,VARCHAR,YES,None,None,None
3,exfSessionId,VARCHAR,YES,None,None,None
4,ipAddress,VARCHAR,YES,None,None,None
5,userId,VARCHAR,YES,None,None,None
6,userPrincipalName,VARCHAR,YES,None,None,None
7,appDisplayName,VARCHAR,YES,None,None,None


In [11]:
# Display the first 10 rows of the events table
events_sample_df = con.execute("SELECT * FROM events LIMIT 10").df()
events_sample_df

,time,operationName,category,exfSessionId,ipAddress,userId,userPrincipalName,appDisplayName
0,2024-07-04T01:29:39.0447402Z,Update user,AuditLogs,510aa855-0fe7-4620-8c4d-72b112626c28,13.91.147.72,None,None,None
1,2024-07-04T07:22:35.4470100Z,Update application,AuditLogs,2f1871ef-5b8f-4344-9f94-f9540143f9bc,None,a41d7eeb-8e40-4878-990f-98342fdd5abb,lukasz.drozdz@exaforce.com,demo-dev
2,2024-07-17T00:50:59.9363246Z,Update StsRefreshTokenValidFrom Timestamp,AuditLogs,f7d994ab-b15e-4478-b667-578a636cf5c3,None,deab9db2-36f5-45b7-b6e1-74e5e8ba9594,magrawal@exaforce.com,None
3,2024-07-17T00:52:41.9992601Z,Update user,AuditLogs,742d1d66-7963-4924-837d-f9d96a20c1fa,None,deab9db2-36f5-45b7-b6e1-74e5e8ba9594,magrawal@exaforce.com,None
4,2024-07-17T13:20:40.3251811Z,Update user,AuditLogs,1da4bb52-426f-4f72-8cad-05755d22b1c5,None,1258c479-653f-4efc-b074-0615d08a06ef,wbeasley@exaforce.com,None
5,2024-07-18T13:11:55.9907608Z,Update service principal,AuditLogs,3f8f8dc9-ce99-4037-8654-d97fec0076ce,None,f03c445a-4270-4fc6-b9a2-a647b428cb5b,krzysztof.kwapisiewicz@exaforce.com,kwapik-dev
6,2024-07-18T13:11:50.8651523Z,Update application,AuditLogs,30d151af-a395-4307-b33f-d13b67f690bd,None,f03c445a-4270-4fc6-b9a2-a647b428cb5b,krzysztof.kwapisiewicz@exaforce.com,kwapik-dev
7,2024-07-18T13:12:02.2129159Z,Update application,AuditLogs,7ef8897f-4c16-462f-9e2e-1f9c85a75418,None,f03c445a-4270-4fc6-b9a2-a647b428cb5b,krzysztof.kwapisiewicz@exaforce.com,kwapik-dev
8,2024-07-18T13:28:39.2316101Z,Update application,AuditLogs,2e5236cd-3113-4346-a629-75899721f3c2,None,f03c445a-4270-4fc6-b9a2-a647b428cb5b,krzysztof.kwapisiewicz@exaforce.com,kwapik-dev
9,2024-07-18T13:29:59.8314375Z,Update application,AuditLogs,89ebdb9e-7577-4aac-a01b-7a7bd11c391d,None,f03c445a-4270-4fc6-b9a2-a647b428cb5b,krzysztof.kwapisiewicz@exaforce.com,kwapik-dev


In [12]:
# Verify the schema of the sessions table
sessions_schema_df = con.execute("DESCRIBE SELECT * FROM sessions").df()
sessions_schema_df

,column_name,column_type,null,key,default,extra
0,exfSessionId,VARCHAR,YES,None,None,None
1,session_start,TIMESTAMP,YES,None,None,None
2,session_end,TIMESTAMP,YES,None,None,None
3,userId,VARCHAR,YES,None,None,None
4,userPrincipalName,VARCHAR,YES,None,None,None


In [13]:
# Display the first 10 rows of the sessions table
sessions_sample_df = con.execute("SELECT * FROM sessions LIMIT 10").df()
sessions_sample_df

,exfSessionId,session_start,session_end,userId,userPrincipalName
0,f7d994ab-b15e-4478-b667-578a636cf5c3,2024-07-17 00:50:59.935324,2024-07-17 00:50:59.936324,deab9db2-36f5-45b7-b6e1-74e5e8ba9594,magrawal@exaforce.com
1,742d1d66-7963-4924-837d-f9d96a20c1fa,2024-07-17 00:52:41.999260,2024-07-17 00:52:41.999260,deab9db2-36f5-45b7-b6e1-74e5e8ba9594,magrawal@exaforce.com
2,2e5236cd-3113-4346-a629-75899721f3c2,2024-07-18 13:28:39.149599,2024-07-18 13:28:39.231610,f03c445a-4270-4fc6-b9a2-a647b428cb5b,krzysztof.kwapisiewicz@exaforce.com
3,f635bc81-f30c-4413-b862-fa58ad1a3428,2024-07-18 17:42:51.786596,2024-07-18 17:42:51.786596,10e44428-4f3e-4029-911f-e004d2655744,smoy@exaforce.com
4,8cd9e7f0-21b5-4eb5-b132-fb3598333f47,2024-07-30 13:49:43.193218,2024-07-30 13:49:43.193218,cafee0c9-f794-4771-98eb-951abed86970,marek@exaforce.com
5,df13c2f5-c38b-40b9-a1c5-5bfaaf79bc43,2024-07-30 15:31:06.005806,2024-07-30 15:31:06.005806,f92b4790-4fb1-4346-95ac-edf5002d40a0,lberanek@exaforce.com
6,2d68c398-a9f8-45cd-8aa2-b0264dec46a7,2024-07-09 08:35:23.293387,2024-07-09 08:35:23.293387,None,None
7,d3734f5c-00fd-4a3d-b97a-08efd6185730,2024-07-09 13:49:24.181471,2024-07-09 13:49:24.181471,a41d7eeb-8e40-4878-990f-98342fdd5abb,lukasz.drozdz@exaforce.com
8,851b408b-f784-4817-a146-7db4f7f220a1,2024-07-09 16:59:06.670414,2024-07-09 16:59:06.670414,None,None
9,972ee4a0-ce9d-4bac-b407-328e3f3516d5,2024-07-09 17:20:20.253181,2024-07-09 17:20:20.253181,None,None


## Step 6: Close the DuckDB Connection
Finally, we close the DuckDB connection.

In [14]:
# Close the DuckDB connection
con.close()